# Markowitz Efficient Frontier

## 1. Imports

In [12]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
from tqdm import tqdm
from fetchData import fetch_raw_data_yf, get_matrices

## 2. Fetch Data

### Get all Nasdaq Stocks

In [2]:
assets= [
    "AAPL",  # Apple Inc.
    "MSFT",  # Microsoft Corporation
    "AMZN",  # Amazon.com Inc.
    "GOOGL", # Alphabet Inc. (Google) Class A
    "GOOG",  # Alphabet Inc. (Google) Class C
    "META",    # Meta Platforms Inc (formerly Facebook)
    "TSLA",  # Tesla Inc
    "UA", # Berkshire Hathaway Inc. Class B
    "JPM",   # JPMorgan Chase & Co.
    "V",     # Visa Inc.
    "JNJ",   # Johnson & Johnson
    "WMT",   # Walmart Inc.
    "PG",    # Procter & Gamble Co.
    "UNH",   # UnitedHealth Group Inc.
    "MA",    # Mastercard Inc.
    "NVDA",  # NVIDIA Corporation
    "HD",    # Home Depot Inc.
    "BAC",   # Bank of America Corp
    "DIS",   # Walt Disney Co
    "PYPL",  # PayPal Holdings
    "VZ",    # Verizon Communications Inc.
    "ADBE",  # Adobe Inc.
    "CMCSA", # Comcast Corporation
    "NFLX",  # Netflix Inc.
    "KO",    # Coca-Cola Co
    "NKE",   # NIKE Inc.
    "PFE",   # Pfizer Inc.
    "MRK",   # Merck & Co., Inc.
    "PEP",   # PepsiCo, Inc.
    "T",     # AT&T Inc.
    "ABT",   # Abbott Laboratories
    "CRM",   # Salesforce.com Inc.
    "ORCL",  # Oracle Corporation
    "ABBV",  # AbbVie Inc.
    "CSCO",  # Cisco Systems, Inc.
    "INTC",  # Intel Corporation
    "TMO",   # Thermo Fisher Scientific Inc.
    "XOM",   # Exxon Mobil Corporation
    "ACN",   # Accenture plc
    "LLY",   # Eli Lilly and Company
    "COST",  # Costco Wholesale Corporation
    "MCD",   # McDonald's Corp
    "DHR",   # Danaher Corporation
    "MDT",   # Medtronic plc
    "NEE",   # NextEra Energy, Inc.
    "BMY",   # Bristol-Myers Squibb Company
    "QCOM",  # Qualcomm Inc
    "CVX",   # Chevron Corporation
    "WFC",   # Wells Fargo & Co
    "LMT"    # Lockheed Martin Corporation
]
assets

['AAPL',
 'MSFT',
 'AMZN',
 'GOOGL',
 'GOOG',
 'META',
 'TSLA',
 'UA',
 'JPM',
 'V',
 'JNJ',
 'WMT',
 'PG',
 'UNH',
 'MA',
 'NVDA',
 'HD',
 'BAC',
 'DIS',
 'PYPL',
 'VZ',
 'ADBE',
 'CMCSA',
 'NFLX',
 'KO',
 'NKE',
 'PFE',
 'MRK',
 'PEP',
 'T',
 'ABT',
 'CRM',
 'ORCL',
 'ABBV',
 'CSCO',
 'INTC',
 'TMO',
 'XOM',
 'ACN',
 'LLY',
 'COST',
 'MCD',
 'DHR',
 'MDT',
 'NEE',
 'BMY',
 'QCOM',
 'CVX',
 'WFC',
 'LMT']

In [3]:
raw_data, asset_errors, max_combination= fetch_raw_data_yf(assets)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Omitted assets( 0 ):  []
Time to fetch data: 7.12 seconds


## 3. Mean, Volatility and Covariance

In [4]:
annualized_returns = get_matrices(raw_data, max_combination ,None)[0][2]
cov = get_matrices(raw_data, max_combination ,None)[0][1]
names = get_matrices(raw_data, max_combination ,None)[0][0]
volatility = np.sqrt(np.diag(cov)) #standard deviation

risk_free_rate=0 
sharpe_ratios = (annualized_returns - risk_free_rate) / volatility

In [5]:
hover_texts = [
    f"{ticker} <br>Volatility: {vol:.2f} <br>Returns: {ret:.2%} <br>Sharpe Ratio: {sr:.2f}"
    for ticker, vol, ret, sr in zip(names, volatility, annualized_returns, sharpe_ratios)
]

fig = go.Figure(data=go.Scatter(
    x=volatility, 
    y=annualized_returns, 
    mode='markers',
    hoverinfo='text',
    hovertext=hover_texts,
    marker=dict(color=sharpe_ratios, colorscale = 'RdBu', size=6, line=dict(width=1), colorbar=dict(title="Sharpe<br>Ratio")
    )
))

fig.update_layout(
    title='Markowitz Mean Varience Model',
    xaxis_title='Volatility (Standard Deviation)',
    yaxis_title='Annualized Returns',
)

fig.show()


In [6]:
portfolio_size = 10
number_of_portfolios = 1000

mean_variance_pairs = []

for i in range(number_of_portfolios):
    assets = np.random.choice(list(raw_data.columns), portfolio_size, replace=False)
    weights = np.random.rand(portfolio_size)
    weights = weights/sum(weights)

    portfolio_E_Variance = 0
    portfolio_E_Return = 0
    for i in range(len(assets)):
        portfolio_E_Return += weights[i] * annualized_returns.loc[assets[i]]
        for j in range(len(assets)):
            portfolio_E_Variance += weights[i] * weights[j] * cov.loc[assets[i], assets[j]]
            
    mean_variance_pairs.append([portfolio_E_Return, portfolio_E_Variance])

In [7]:
#-- Plot the risk vs. return of randomly generated portfolios
#-- Convert the list from before into an array for easy plotting
mean_variance_pairs = np.array(mean_variance_pairs)

risk_free_rate=0 #-- Include risk free rate here

fig = go.Figure()
fig.add_trace(go.Scatter(x=mean_variance_pairs[:,1]**0.5, y=mean_variance_pairs[:,0], 
    marker=dict(color=(mean_variance_pairs[:,0]-risk_free_rate)/(mean_variance_pairs[:,1]**0.5), 
                size=5,
                line=dict(width=1),
                colorbar=dict(title="Sharpe<br>Ratio")
                ), 
    mode='markers'))
fig.update_layout( 
    xaxis=dict(title='Volatility (Standard Deviation)'),
    yaxis=dict(title='Annualised Returns'),
    title='Sample of Random Portfolios')

In [8]:
#-- Create random portfolio weights and indexes
#- How many assests in the portfolio
n_assets = 10

mean_variance_pairs = []
weights_list=[]
tickers_list=[]

for i in tqdm(range(100000)):
    next_i = False
    while True:
        #- Choose assets randomly without replacement
        assets = np.random.choice(list(raw_data.columns), n_assets, replace=False)
        #- Choose weights randomly ensuring they sum to one
        weights = np.random.rand(n_assets)
        weights = weights/sum(weights)

        #-- Loop over asset pairs and compute portfolio return and variance
        portfolio_E_Variance = 0
        portfolio_E_Return = 0
        for i in range(len(assets)):
            portfolio_E_Return += weights[i] * annualized_returns.loc[assets[i]]
            for j in range(len(assets)):
                portfolio_E_Variance += weights[i] * weights[j] * cov.loc[assets[i], assets[j]]

        #-- Skip over dominated portfolios
        for R,V in mean_variance_pairs:
            if (R > portfolio_E_Return) & (V < portfolio_E_Variance):
                next_i = True
                break
        if next_i:
            break

        #-- Add the mean/variance pairs to a list for plotting
        mean_variance_pairs.append([portfolio_E_Return, portfolio_E_Variance])
        weights_list.append(weights)
        tickers_list.append(assets)
        break


100%|██████████| 100000/100000 [00:44<00:00, 2225.36it/s]


In [11]:
len(mean_variance_pairs)

172

In [10]:
#-- Plot the risk vs. return of randomly generated portfolios
#-- Convert the list from before into an array for easy plotting
mean_variance_pairs = np.array(mean_variance_pairs)

risk_free_rate=0 #-- Include risk free rate here

fig = go.Figure()
fig.add_trace(go.Scatter(x=mean_variance_pairs[:,1]**0.5, y=mean_variance_pairs[:,0], 
                      marker=dict(color=(mean_variance_pairs[:,0]-risk_free_rate)/(mean_variance_pairs[:,1]**0.5), 
                                  showscale=True, 
                                  size=7,
                                  line=dict(width=1),
                                  colorscale="RdBu",
                                  colorbar=dict(title="Sharpe<br>Ratio")
                                 ), 
                      mode='markers',
                      text=[str(np.array(tickers_list[i])) + "<br>" + str(np.array(weights_list[i]).round(2)) for i in range(len(tickers_list))]))
fig.update_layout(template='plotly_white',
    xaxis=dict(title='Annualised Risk (Volatility)'),
    yaxis=dict(title='Annualised Return'),
    title='Sample of Random Portfolios'
    )